In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
from io import BytesIO
import random
import shutil
import itertools as it

In [2]:
def loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

In [3]:
files = glob('/home/mephistopheies/storage2/data/camera-model-id/pseudo_labels/voters_phase_2/*.csv')

votes = defaultdict(list)
for fname in tqdm_notebook(files):
    df = pd.read_csv(fname)    
    for ix, r in df.iterrows():
        votes[r['fname']].append(r['camera'])

In [4]:
conf = {}

for k, v in votes.items():
    conf[k] = len(set(v))

In [5]:
pd.Series(list(conf.values())).value_counts()

1    2409
2     156
3      52
4      20
5       2
6       1
dtype: int64

In [98]:
joints = defaultdict(float)
for fname, v in votes.items():
        
    for c1, c2 in it.combinations(v, 2):
        joints[tuple(sorted([c1, c2]))] += 1

In [105]:
joints = dict([(k, v/(len(votes)*15)) for (k, v) in joints.items()])

In [106]:
classes = list(df['camera'].unique())

In [108]:
m = np.zeros((len(classes), len(classes)))
for c1, c2 in it.combinations(classes, 2):
    m[classes.index(c1), classes.index(c2)] = joints[tuple(sorted([c1, c2]))]

In [34]:
with open('/home/mephistopheies/storage2/data/camera-model-id/tmp/p_test_985_m1_u0.pkl', 'rb') as f:
    p_test1 = pickle.load(f)

with open('/home/mephistopheies/storage2/data/camera-model-id/tmp/p_test_985_m2_u0.pkl', 'rb') as f:
    p_test2 = pickle.load(f)

In [35]:
pred = {}
pred_manip = {}
pred_unalt = {}

for fname in p_test1.keys():
    c1 = sorted(p_test1[fname], key=lambda t: t[1], reverse=True)[0][0]
    c2 = sorted(p_test2[fname], key=lambda t: t[1], reverse=True)[0][0]
    
    if c1 != c2:
        pred[fname] = (c1, c2)
        if '_unalt' in fname:
            pred_unalt[fname] = (c1, c2)
        else:
            pred_manip[fname] = (c1, c2)

In [59]:
# wins = []

# for fname in pred.keys():
#     tmp = zip(sorted(p_test1[fname], key=lambda t: t[1], reverse=True), 
#               sorted(p_test2[fname], key=lambda t: t[1], reverse=True))
    
#     for ix, ((c1, p1), (c2, p2)) in enumerate(tmp):
#         if c1 == c2:
#             wins.append((ix, ((c1, p1), (c2, p2))))
#             break